In [127]:
%load_ext autoreload
%autoreload 2
from app_tricount import fetch_data, build_data, format_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [129]:

data, dict = fetch_data()
dict = dict[['Postes', 'Catégories']].set_index('Catégories').to_dict()['Postes']
dframe = build_data(data)
tables = format_data(dframe)
# result = concat_data(tables, dict)


    # --------------------------------- #
    # Connection to Google Sheet        #
    # and extract Spreadsheets          #
    #  - exported tricount data         #
    #  - category dictonnary            #
    # --------------------------------- #
    
 -- data fetched
(1023, 16)
Index(['Titre', 'Montant', 'Devise', 'Taux de change',
       'Montant dans la devise du tricount (EUR)', 'Type de transaction',
       'Catégorie', 'Payé par', 'Payé par Commun', 'Payé par Lucie',
       'Payé par Vincent', 'Impacté à Commun', 'Impacté à Lucie',
       'Impacté à Vincent', 'Date & heure', 'URL des images'],
      dtype='object')
 -- dict fetched
          Catégories  P            Postes
0       Alimentation  Q         Quotidien
1           Shopping  A            Achats
2           Logement  A            Achats
3    Loyer & charges  Q         Quotidien
4     Divertissement  E             Extra
5              Sport  L            Loisir
6   Restaurant & bar  E             Extra
7     Invest

In [131]:
dict['Alimentation']

'Quotidien'

In [89]:
result = {}
for table in tables.keys():
    df = tables[table].reset_index()
    df['Poste'] = df['Catégorie'].apply(lambda category: dict[category])
    df = df.set_index('Poste')[['Total', 'Lucie', 'Vincent']]
    df = df.groupby('Poste').sum()
    df = df.transpose()

    for poste in dict['Postes'].unique().tolist():
        if poste not in df.columns:
            df[poste] = 0

    df['Revenus'] = df['Rentrée d\'argent']
    df['Dépenses'] = df['Quotidien'] + df['Loisir'] + df['Extra'] + df['Achats']
    df['Reste à vivre'] = df['Dépenses'] - df['Revenus'] 
    df['Reste à vivre %'] = df['Reste à vivre'] / df['Revenus'] * 100
    df['Capital investi'] = df['Investissement']
    df['Capital investi %'] = df['Capital investi'] / df['Revenus'] * 100
    df['Epargne'] = df['Reste à vivre'] - df['Capital investi']
    df['Epargne %'] = df['Epargne'] / df['Revenus'] * 100
    df = df.transpose()
    print(df)
    result[table] = df



# result

                     Total    Lucie  Vincent
Poste                                       
Achats              345.00   -92.50   437.50
Extra               -81.00   -40.50   -40.50
Investissement      106.00     0.00   106.00
Loisir              -81.02   -40.51   -40.51
Quotidien         -1243.98  -846.14  -397.84
Rentrée d'argent      0.00     0.00     0.00
Revenus               0.00     0.00     0.00
Dépenses          -1061.00 -1019.65   -41.35
Reste à vivre     -1061.00 -1019.65   -41.35
Reste à vivre %       -inf     -inf     -inf
Capital investi     106.00     0.00   106.00
Capital investi %      inf      NaN      inf
Epargne           -1167.00 -1019.65  -147.35
Epargne %             -inf     -inf     -inf
                     Total    Lucie  Vincent
Poste                                       
Achats              345.00   -92.50   437.50
Extra               -81.00   -40.50   -40.50
Investissement      106.00     0.00   106.00
Loisir              -81.02   -40.51   -40.51
Quotidien 